# AI-PSCI-014: Building an End-to-End Drug Discovery Pipeline

**AI in Pharmaceutical Sciences: Bench to Bedside**  
VCU School of Pharmacy | VIP Program | Spring 2026

---

**Week 7 | Module: Prototyping | Estimated Time: 90-120 minutes**

**Prerequisites**: AI-PSCI-001 through AI-PSCI-013

**GPU Recommended**: For ESMFold and ESM-1v predictions.

---

## 🎯 Learning Objectives

After completing this talktorial, you will be able to:

1. Design and implement an integrated computational pipeline
2. Chain multiple AI tools (ESMFold, ESM-1v) in a coherent workflow
3. Create comprehensive mutation analysis reports
4. Document methodology for reproducibility
5. Synthesize insights from multiple computational approaches

---

## 📚 Background

### The Power of Integrated Pipelines

In the previous talktorials, you've learned individual tools:
- **AI-PSCI-009**: Fetching sequences from UniProt
- **AI-PSCI-010**: Visualizing structures with py3Dmol
- **AI-PSCI-012**: Predicting structures with ESMFold
- **AI-PSCI-013**: Scoring mutations with ESM-1v

Now it's time to **integrate these tools** into a cohesive pipeline that answers real drug discovery questions.

### Pipeline Architecture

```
┌─────────────────┐
│  1. Sequence    │  ← UniProt fetch
│     Input       │
└────────┬────────┘
         │
         ▼
┌─────────────────┐
│  2. Generate    │  ← Create mutation variants
│     Variants    │
└────────┬────────┘
         │
    ┌────┴────┐
    ▼         ▼
┌───────┐ ┌───────┐
│ESMFold│ │ESM-1v │  ← Parallel predictions
│pLDDT  │ │ LLR   │
└───┬───┘ └───┬───┘
    └────┬────┘
         ▼
┌─────────────────┐
│  3. Integrate   │  ← Combine structural + evolutionary
│     Results     │
└────────┬────────┘
         ▼
┌─────────────────┐
│  4. Visualize   │  ← py3Dmol, matplotlib
│  & Report       │
└─────────────────┘
```

### Why Integration Matters

Each tool provides different insights:
- **ESMFold pLDDT**: Structural confidence — does the mutation disrupt folding?
- **ESM-1v LLR**: Evolutionary constraint — is this mutation unusual?

Combining them gives a more complete picture than either alone.

### Midterm Context

This talktorial serves as **midterm preparation**. You're demonstrating mastery of:
- Data acquisition and preparation
- AI tool integration
- Results synthesis and communication

---

## 🛠️ Setup

Run this cell to install required packages. **This takes 3-5 minutes.**

In [ ]:
#@title 🛠️ Install Packages (3-5 min)
import sys

# Check for GPU
try:
    gpu_info = !nvidia-smi --query-gpu=name --format=csv,noheader
    if gpu_info and gpu_info[0] and 'NVIDIA' in gpu_info[0]:
        print(f"✅ GPU detected: {gpu_info[0]}")
        HAS_GPU = True
    else:
        print("⚠️ No GPU detected. Predictions will be slower.")
        HAS_GPU = False
except Exception:
    print("⚠️ Could not detect GPU.")
    HAS_GPU = False

# Install PyTorch
print("\n📦 Installing PyTorch...")
if HAS_GPU:
    !pip install torch torchvision --index-url https://download.pytorch.org/whl/cu121 -q
else:
    !pip install torch torchvision -q

# Install ESM (for ESM-1v)
print("📦 Installing ESM (fair-esm)...")
!pip install fair-esm -q

# Install other packages
print("📦 Installing analysis packages...")
!pip install biopython py3Dmol matplotlib seaborn pandas numpy requests -q

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

print("\n✅ All packages installed!")

Import required libraries:

In [ ]:
#@title 📦 Import Libraries
import torch
import esm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import py3Dmol
import requests
from pathlib import Path
from typing import List, Dict, Tuple, Optional
from datetime import datetime
import time
import json

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🖥️ Using device: {device}")

# Set display options
pd.set_option('display.max_columns', 15)
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['figure.dpi'] = 100

print("✅ All libraries imported!")

---

## 🎯 Target Configuration

Select your drug target. This selection determines which protein you'll analyze throughout the pipeline.

In [ ]:
#@title 🎯 Select Your Drug Target
TARGET = "DHFR" #@param ["DHFR", "ABL1", "EGFR", "AChE", "COX-2", "DPP-4"]

TARGET_CONFIG = {
    "DHFR": {
        "pdb": "1RX1", "uniprot": "P0ABQ4", "chembl": "CHEMBL202",
        "drug": "Trimethoprim", "organism": "E. coli",
        "mutations": ["L28R", "P21L", "A26T", "W30R", "I94L"],
        "mutation_context": "Trimethoprim resistance mutations",
        "expected_length": 159
    },
    "ABL1": {
        "pdb": "1IEP", "uniprot": "P00519", "chembl": "CHEMBL1862",
        "drug": "Imatinib", "organism": "Human",
        "mutations": ["T315I", "E255K", "Y253H", "G250E", "M351T"],
        "mutation_context": "Imatinib resistance in CML",
        "expected_length": 1130, "domain_start": 242, "domain_end": 493
    },
    "EGFR": {
        "pdb": "1M17", "uniprot": "P00533", "chembl": "CHEMBL203",
        "drug": "Erlotinib", "organism": "Human",
        "mutations": ["T790M", "L858R", "C797S", "G719S", "L861Q"],
        "mutation_context": "Drug resistance and activating mutations",
        "expected_length": 1210, "domain_start": 712, "domain_end": 979
    },
    "AChE": {
        "pdb": "4EY7", "uniprot": "P22303", "chembl": "CHEMBL220",
        "drug": "Donepezil", "organism": "Human",
        "mutations": ["Y337A", "F338A", "W286A", "E202Q", "S203A"],
        "mutation_context": "Catalytic gorge mutations",
        "expected_length": 614
    },
    "COX-2": {
        "pdb": "3LN1", "uniprot": "P35354", "chembl": "CHEMBL230",
        "drug": "Celecoxib", "organism": "Human",
        "mutations": ["V523I", "S530A", "Y385F", "R120A", "H90Q"],
        "mutation_context": "Selectivity and catalytic mutations",
        "expected_length": 604
    },
    "DPP-4": {
        "pdb": "1X70", "uniprot": "P27487", "chembl": "CHEMBL284",
        "drug": "Sitagliptin", "organism": "Human",
        "mutations": ["S630A", "E205A", "E206A", "Y547A", "R125A"],
        "mutation_context": "Catalytic and binding site mutations",
        "expected_length": 766
    }
}

config = TARGET_CONFIG[TARGET]
print(f"✅ Target: {TARGET}")
print(f"   PDB: {config['pdb']} | UniProt: {config['uniprot']}")
print(f"   Reference Drug: {config['drug']}")
print(f"   Organism: {config['organism']}")
print(f"\n🧬 Mutations to analyze: {', '.join(config['mutations'])}")
print(f"   Context: {config['mutation_context']}")

# Create output directory
output_dir = Path(f"pipeline_{TARGET}")
output_dir.mkdir(exist_ok=True)
print(f"\n📁 Output directory: {output_dir}")

---

## 🔬 Guided Inquiry 1: Building the Pipeline Framework

### Context

Before running predictions, we need to set up a clean framework that will:
1. Store results from multiple tools
2. Track progress through the pipeline
3. Handle errors gracefully

### Your Task

Using your AI assistant, write code to:

1. Create a `Pipeline` class or data structure to track results
2. Add a `fetch_sequence` function using UniProt API
3. Add a `generate_variants` function to create mutant sequences
4. Initialize the pipeline with your target's mutations

💡 **Prompting Tips**:
- Ask: "How do I design a Python class to track results from multiple analysis steps?"
- Consider using a dictionary or dataclass to store per-mutation results
- Remember to handle kinase domain extraction for large proteins

### Verification

After running your code, confirm:
- [ ] Wild-type sequence is fetched successfully
- [ ] All variant sequences are generated correctly
- [ ] Each mutation position is verified against the sequence

📓 **Lab Notebook**: Document your pipeline architecture. What data will you track for each mutation?

In [ ]:
# Your code here


---

## 🔬 Guided Inquiry 2: ESM-1v Evolutionary Analysis

### Context

Now we'll add evolutionary analysis using ESM-1v. This tells us whether mutations are evolutionarily disfavored (potentially deleterious).

### Your Task

Using your AI assistant, write code to:

1. Load ESM-1v model (we'll use 1 model for speed)
2. Create a function to score a single mutation
3. Score all mutations in the pipeline
4. Store results in the pipeline object

💡 **Prompting Tips**:
- Refer back to AI-PSCI-013 for ESM-1v patterns
- Ask: "How do I integrate ESM-1v scoring into my existing pipeline class?"
- Use masked marginal scoring (mask one position, predict)

### Verification

After running your code, confirm:
- [ ] Model loads successfully
- [ ] All mutations are scored
- [ ] LLR values are in expected range (-7 to +2)

📓 **Lab Notebook**: Which mutation has the most negative LLR? What does this suggest?

In [ ]:
# Your code here


---

## 🔬 Guided Inquiry 3: ESMFold Structural Analysis

### Context

Now we add structural predictions using ESMFold. This tells us whether mutations affect protein folding (via pLDDT changes).

### Your Task

Using your AI assistant, write code to:

1. Create a function to call ESMFold API
2. Predict structure for wild-type sequence
3. Predict structures for all mutant sequences
4. Extract and compare pLDDT scores

💡 **Prompting Tips**:
- Refer back to AI-PSCI-012 for ESMFold API patterns
- ESMFold API: `https://api.esmatlas.com/foldSequence/v1/pdb/`
- pLDDT is stored in B-factor column (0-1 scale, multiply by 100)

### Verification

After running your code, confirm:
- [ ] Wild-type structure is predicted
- [ ] All mutant structures are predicted
- [ ] pLDDT values are in 0-100 range

📓 **Lab Notebook**: Which mutation shows the largest pLDDT change? What might this indicate?

In [ ]:
# Your code here


---

## 🔬 Guided Inquiry 4: Integrating Results

### Context

Now comes the key step: **integrating** results from both analyses to create a comprehensive mutation assessment.

### Your Task

Using your AI assistant, write code to:

1. Create a summary DataFrame combining all results
2. Add a composite "risk score" based on ESM-1v and ESMFold
3. Classify mutations into risk categories
4. Identify the most concerning mutations

💡 **Prompting Tips**:
- Ask: "How can I combine evolutionary (LLR) and structural (pLDDT) scores into a single metric?"
- Consider normalizing scores before combining
- Think about which score should have more weight

### Verification

After running your code, confirm:
- [ ] Summary table includes all metrics
- [ ] Risk scores are calculated
- [ ] Mutations are ranked by risk

📓 **Lab Notebook**: Do the ESM-1v and ESMFold results agree? If not, what might explain the disagreement?

In [ ]:
# Your code here


---

## 🔬 Guided Inquiry 5: Visualization Dashboard

### Context

Visualization is crucial for communicating results. We'll create a multi-panel dashboard showing all aspects of our analysis.

### Your Task

Using your AI assistant, write code to:

1. Create a 2x2 figure with multiple subplots
2. Panel 1: ESM-1v LLR bar chart
3. Panel 2: ESMFold pLDDT comparison
4. Panel 3: Risk score ranking
5. Panel 4: Summary statistics

💡 **Prompting Tips**:
- Ask: "How do I create a publication-quality multi-panel figure?"
- Use consistent color schemes across panels
- Color-code by risk category

### Verification

After running your code, confirm:
- [ ] All panels are visible and labeled
- [ ] Colors are consistent and meaningful
- [ ] Figure is saved to output directory

📓 **Lab Notebook**: What insights emerge from visualizing the data this way?

In [ ]:
# Your code here


---

## 🔬 Guided Inquiry 6: Generating the Final Report

### Context

The final step is creating a comprehensive report that documents everything we've done. This is crucial for reproducibility and communication.

### Your Task

Using your AI assistant, write code to:

1. Generate a markdown report summarizing the analysis
2. Include methodology, results, and conclusions
3. List all output files generated
4. Add timestamp and version information

💡 **Prompting Tips**:
- Ask: "How do I generate a markdown report in Python?"
- Include both human-readable summary and machine-readable data
- Add methodology section for reproducibility

### Verification

After running your code, confirm:
- [ ] Report is generated and saved
- [ ] All sections are complete
- [ ] Report is readable and informative

📓 **Lab Notebook**: Summarize your key findings in 2-3 sentences.

In [ ]:
# Your code here


---

## ✅ Checkpoint

Before moving on to the next talktorial, confirm you can:

- [ ] Design and implement an integrated pipeline
- [ ] Chain ESM-1v and ESMFold analyses
- [ ] Combine metrics into a composite risk score
- [ ] Create visualizations for multiple data types
- [ ] Generate reproducible reports

### Your lab notebook should include:

- [ ] Pipeline architecture diagram
- [ ] All mutation results with risk scores
- [ ] Key findings for your target
- [ ] Reflection on method agreement/disagreement

---

## 🤔 Reflection Questions

Answer these in your lab notebook:

1. **Method Agreement**: Did ESM-1v and ESMFold results agree? For which mutations did they disagree, and what might explain this?

2. **Pipeline Design**: What would you add to this pipeline if you had more time? Consider: experimental data integration, additional AI tools, or different visualization approaches.

3. **Clinical Translation**: How could a pharmaceutical company use this pipeline in drug development? What additional validation would be needed before clinical decisions?

---

## 📖 Further Reading

- [ESM-1v Paper](https://www.biorxiv.org/content/10.1101/2021.07.09.450648v2) - Variant effect prediction
- [ESMFold Paper](https://www.science.org/doi/10.1126/science.ade2574) - Language model structure prediction
- [Pipeline Design Patterns](https://www.nature.com/articles/s41592-021-01252-x) - Computational biology pipelines
- [Reproducible Research](https://journals.plos.org/ploscompbiol/article?id=10.1371/journal.pcbi.1003285) - Best practices

---

## 🔗 Connection to Research

This integrated pipeline represents a realistic workflow used in pharmaceutical research:

- **Drug Resistance Surveillance**: Rapidly assess emerging mutations in patient populations
- **Protein Engineering**: Guide design of more stable or active protein variants
- **Clinical Trial Design**: Prioritize which mutations to test experimentally
- **Regulatory Submissions**: Provide computational evidence for drug safety/efficacy

The combination of evolutionary (ESM-1v) and structural (ESMFold) analyses is becoming standard practice. Tools like AlphaMissense and ProteinGym build on similar principles.

In the next module, you'll learn molecular docking (AI-PSCI-015) to add drug-protein interaction analysis to your toolkit.

---

*AI-PSCI-014 Complete. Proceed to AI-PSCI-015: Molecular Docking with AutoDock Vina.*